In [ ]:
# 1. Carga y unión de datos
import pandas as pd
import os

# Definir ruta base
base_path = r"D:\HP\Downloads\Ganancias de los eSports"

# Cargar archivos CSV
df_players = pd.read_csv(os.path.join(base_path, "highest_earning_players.csv"))
df_teams = pd.read_csv(os.path.join(base_path, "highest_earning_teams.csv"))
df_country_codes = pd.read_csv(os.path.join(base_path, "country-and-continent-codes-list.csv"))

# Explorar brevemente
print(df_players.head())
print(df_teams.head())
print(df_country_codes.head())


In [1]:
# 2. Análisis Exploratorio de Datos (EDA)

# Información general y tipos de datos
print(df_players.info())

# Estadísticas descriptivas de variables numéricas
print(df_players.describe())

# Revisión de valores faltantes
print(df_players.isnull().sum())

import matplotlib.pyplot as plt
import seaborn as sns

# Distribución de la variable objetivo
sns.histplot(df_players['TotalUSDPrize'], bins=50, kde=True)
plt.title("Distribución de Ganancias Totales")
plt.xlabel("Ganancias Totales (USD)")
plt.ylabel("Frecuencia")
plt.show()

# Análisis de variables categóricas: actualizar nombres de columnas según tu dataset
categorical_cols = ['Game', 'Genre', 'CountryCode']

for col in categorical_cols:
    print(f"Valores únicos en {col}: {df_players[col].nunique()}")
    plt.figure(figsize=(8,6))
    sns.countplot(y=df_players[col], order=df_players[col].value_counts().index)
    plt.title(f"Distribución de {col}")
    plt.xlabel("Cantidad de jugadores")
    plt.ylabel(col)
    plt.show()


NameError: name 'df_players' is not defined